In [19]:
df=spark.sql("""
    select date(now()-interval '1' day) as date,
sum(if(a.loan_type !='ROLLOVER',a.amount,0)) as total_issue_amount,sum(b.principal_paid) as total_paid_off_principle,sum(if(a.loan_type !='ROLLOVER',a.amount,0))-sum(b.principal_paid) as loan_balance,
sum(b.principal_paid + b.interest_paid + b.rollover_fee_paid + b.default_paid + b.default_fee_paid + b.service_fee_paid-b.erase_amount-b.coupon_amount) as total_paid_off_amount,

sum(if(a.status in ('CURRENT','OVERDUE','GRACE_PERIOD'),(b.principal_accr + b.interest_accr + b.default_accr + b.default_fee_accr + b.service_fee_accr) -(b.principal_paid + b.interest_paid + b.default_paid + b.default_fee_paid + b.service_fee_paid),0)) as total_outstanding_amount,

sum(if(a.status in ('CURRENT'),(b.principal_accr + b.interest_accr + b.default_accr + b.default_fee_accr + b.service_fee_accr) -(b.principal_paid + b.interest_paid + b.default_paid + b.default_fee_paid + b.service_fee_paid),0)) as total_current_amount,
sum(if(a.status in ('OVERDUE','GRACE_PERIOD') and datediff(now(),date(b.due_date))>=0 and datediff(now(),date(b.due_date))<=3,(b.principal_accr + b.interest_accr + b.default_accr + b.default_fee_accr + b.service_fee_accr) -(b.principal_paid + b.interest_paid + b.default_paid + b.default_fee_paid + b.service_fee_paid),0)) as overdue_3_amount,
sum(if(a.status in ('OVERDUE','GRACE_PERIOD') and datediff(now(),date(b.due_date))>=4 and datediff(now(),date(b.due_date))<=10,(b.principal_accr + b.interest_accr + b.default_accr + b.default_fee_accr + b.service_fee_accr) -(b.principal_paid + b.interest_paid + b.default_paid + b.default_fee_paid + b.service_fee_paid),0)) as overdue_10_amount,
sum(if(a.status in ('OVERDUE','GRACE_PERIOD') and datediff(now(),date(b.due_date))>=11 and datediff(now(),date(b.due_date))<=30,(b.principal_accr + b.interest_accr + b.default_accr + b.default_fee_accr + b.service_fee_accr) -(b.principal_paid + b.interest_paid + b.default_paid + b.default_fee_paid + b.service_fee_paid),0)) as overdue_30_amount,
sum(if(a.status in ('OVERDUE','GRACE_PERIOD') and datediff(now(),date(b.due_date))>=31 and datediff(now(),date(b.due_date))<=60,(b.principal_accr + b.interest_accr + b.default_accr + b.default_fee_accr + b.service_fee_accr) -(b.principal_paid + b.interest_paid + b.default_paid + b.default_fee_paid + b.service_fee_paid),0)) as overdue_60_amount,
sum(if(a.status in ('OVERDUE','GRACE_PERIOD') and datediff(now(),date(b.due_date))>=61 and datediff(now(),date(b.due_date))<=90,(b.principal_accr + b.interest_accr + b.default_accr + b.default_fee_accr + b.service_fee_accr) -(b.principal_paid + b.interest_paid + b.default_paid + b.default_fee_paid + b.service_fee_paid),0)) as overdue_90_amount,
sum(if(a.status in ('OVERDUE','GRACE_PERIOD') and datediff(now(),date(b.due_date))>=91,(b.principal_accr + b.interest_accr + b.default_accr + b.default_fee_accr + b.service_fee_accr) -(b.principal_paid + b.interest_paid + b.default_paid + b.default_fee_paid + b.service_fee_paid),0)) as overdue_more_than_90_amount
from  `banda-etl-s3`.`t_loan_app` a
left  join `banda-etl-s3`.`t_lpay` b
on a.id = b.loan_app_id
where a.status in ('CURRENT','PAID_OFF','OVERDUE','GRACE_PERIOD') and a.product_name='RUPIAHONE' and a.create_time + interval '7' hour >= date '2019-08-27'
""")
df.write.mode("append").orc("s3://rupiahplus-data-warehouse/etl/banda/market/loan_banlace_summary")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
t_admin_audit=spark.sql("""
 select  datediff(from_utc_timestamp(now(), 'Asia/Shanghai'),date(b.due_date)) as dd,
 datediff(date(now()+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour)) as ddd,
 datediff(date(now()+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))
from `banda-etl-s3`.`t_lpay` b
order by id
limit 100

""").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+----+
|  dd| ddd|
+----+----+
|-387|-387|
|-387|-387|
|-387|-387|
|-387|-387|
|-387|-387|
|-387|-387|
|-387|-387|
|-387|-387|
| 344| 344|
|1014|1014|
|1033|1033|
|1021|1021|
|1017|1017|
|1009|1009|
|1009|1009|
|1009|1009|
|1008|1008|
|1008|1008|
|1015|1015|
|1008|1008|
+----+----+
only showing top 20 rows

In [7]:
df=spark.sql("""
select 
date(now()+INTERVAL '7' hour -interval '1' day) as date,
sum(if(a.loan_type !='ROLLOVER',a.amount,0)) as total_issue_amount,sum(b.principal_paid) as total_paid_off_principle,
sum(if(a.loan_type !='ROLLOVER',a.amount,0))-sum(b.principal_paid) as loan_balance,
sum(if(a.status in ('CURRENT','OVERDUE','GRACE_PERIOD') and datediff(date(now()+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour)) <=0,(b.principal_accr) -(b.principal_paid ),0)) as total_current_principle,
sum(if(a.status in ('CURRENT','OVERDUE','GRACE_PERIOD') and datediff(date(now()+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))>=1 and  datediff(date(now()+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))<=3,(b.principal_accr) -(b.principal_paid),0)) as overdue_3_principle,
sum(if(a.status in ('CURRENT','OVERDUE','GRACE_PERIOD') and datediff(date(now()+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))>=4 and  datediff(date(now()+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))<=10,(b.principal_accr ) -(b.principal_paid ),0)) as overdue_10_principle,
sum(if(a.status in ('CURRENT','OVERDUE','GRACE_PERIOD') and datediff(date(now()+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))>=11 and  datediff(date(now()+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))<=30,(b.principal_accr ) -(b.principal_paid ),0)) as overdue_30_principle,
sum(if(a.status in ('CURRENT','OVERDUE','GRACE_PERIOD') and datediff(date(now()+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))>=31 and  datediff(date(now()+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))<=60,(b.principal_accr ) -(b.principal_paid ),0)) as overdue_60_principle,
sum(if(a.status in ('CURRENT','OVERDUE','GRACE_PERIOD') and datediff(date(now()+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))>=61 and  datediff(date(now()+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))<=90,(b.principal_accr ) -(b.principal_paid ),0)) as overdue_90_principle,
sum(if(a.status in ('CURRENT','OVERDUE','GRACE_PERIOD') and datediff(date(now()+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))>=91,(b.principal_accr ) -(b.principal_paid ),0)) as overdue_more_than_90_principle,
sum(b.principal_paid + b.interest_paid + b.rollover_fee_paid + b.default_paid + b.default_fee_paid + b.service_fee_paid-b.erase_amount-b.coupon_amount) as total_paid_off_amount,
sum(if(a.status in ('CURRENT','OVERDUE','GRACE_PERIOD'),(b.principal_accr + b.interest_accr + b.default_accr + b.default_fee_accr + b.service_fee_accr) -(b.principal_paid + b.interest_paid + b.default_paid + b.default_fee_paid + b.service_fee_paid),0)) as total_outstanding_amount,
sum(if(a.status in ('CURRENT','OVERDUE','GRACE_PERIOD') and datediff(date(now()+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour)) <=0,(b.principal_accr + b.interest_accr + b.default_accr + b.default_fee_accr + b.service_fee_accr) -(b.principal_paid + b.interest_paid + b.default_paid + b.default_fee_paid + b.service_fee_paid),0)) as total_current_amount,
sum(if(a.status in ('CURRENT','OVERDUE','GRACE_PERIOD') and datediff(date(now()+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))>=1 and  datediff(date(now()+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))<=3,(b.principal_accr + b.interest_accr + b.default_accr + b.default_fee_accr + b.service_fee_accr) -(b.principal_paid + b.interest_paid + b.default_paid + b.default_fee_paid + b.service_fee_paid),0)) as overdue_3_amount,
sum(if(a.status in ('CURRENT','OVERDUE','GRACE_PERIOD') and datediff(date(now()+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))>=4 and  datediff(date(now()+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))<=10,(b.principal_accr + b.interest_accr + b.default_accr + b.default_fee_accr + b.service_fee_accr) -(b.principal_paid + b.interest_paid + b.default_paid + b.default_fee_paid + b.service_fee_paid),0)) as overdue_10_amount,
sum(if(a.status in ('CURRENT','OVERDUE','GRACE_PERIOD') and datediff(date(now()+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))>=11 and  datediff(date(now()+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))<=30,(b.principal_accr + b.interest_accr + b.default_accr + b.default_fee_accr + b.service_fee_accr) -(b.principal_paid + b.interest_paid + b.default_paid + b.default_fee_paid + b.service_fee_paid),0)) as overdue_30_amount,
sum(if(a.status in ('CURRENT','OVERDUE','GRACE_PERIOD') and datediff(date(now()+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))>=31 and  datediff(date(now()+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))<=60,(b.principal_accr + b.interest_accr + b.default_accr + b.default_fee_accr + b.service_fee_accr) -(b.principal_paid + b.interest_paid + b.default_paid + b.default_fee_paid + b.service_fee_paid),0)) as overdue_60_amount,
sum(if(a.status in ('CURRENT','OVERDUE','GRACE_PERIOD') and datediff(date(now()+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))>=61 and  datediff(date(now()+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))<=90,(b.principal_accr + b.interest_accr + b.default_accr + b.default_fee_accr + b.service_fee_accr) -(b.principal_paid + b.interest_paid + b.default_paid + b.default_fee_paid + b.service_fee_paid),0)) as overdue_90_amount,
sum(if(a.status in ('CURRENT','OVERDUE','GRACE_PERIOD') and datediff(date(now()+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))>=91,(b.principal_accr + b.interest_accr + b.default_accr + b.default_fee_accr + b.service_fee_accr) -(b.principal_paid + b.interest_paid + b.default_paid + b.default_fee_paid + b.service_fee_paid),0)) as overdue_more_than_90_amount
from `banda-etl-s3`.`t_loan_app` a
left  join `banda-etl-s3`.`t_lpay` b
on a.id = b.loan_app_id
where a.status in ('CURRENT','PAID_OFF','OVERDUE','GRACE_PERIOD') and a.product_name='RUPIAHONE' and a.create_time + interval '7' hour >= date '2019-08-25'
""").show()
# df.write.mode("append").orc("s3://rupiahplus-data-warehouse/etl/banda/market/loan_banlace_summary")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+--------------------+------------------------+-------------------+-----------------------+-------------------+--------------------+--------------------+--------------------+--------------------+------------------------------+---------------------+------------------------+--------------------+-------------------+-------------------+-------------------+--------------------+--------------------+---------------------------+
|      date|  total_issue_amount|total_paid_off_principle|       loan_balance|total_current_principle|overdue_3_principle|overdue_10_principle|overdue_30_principle|overdue_60_principle|overdue_90_principle|overdue_more_than_90_principle|total_paid_off_amount|total_outstanding_amount|total_current_amount|   overdue_3_amount|  overdue_10_amount|  overdue_30_amount|   overdue_60_amount|   overdue_90_amount|overdue_more_than_90_amount|
+----------+--------------------+------------------------+-------------------+-----------------------+-------------------+------

In [6]:
df=spark.sql("""
SELECT from_utc_timestamp(now(), 'Asia/Shanghai')
""").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------------------------------------+
|from_utc_timestamp(current_timestamp(), Asia/Shanghai)|
+------------------------------------------------------+
|                                  2020-02-14 11:37:...|
+------------------------------------------------------+